# NSight System trace analysis

Parsing Nsight System trace exported as JSON

In [1]:
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
from matplotlib import pyplot as plt 
import re

## Functions

In [2]:
# Return True if the row value contains any of event name patterns
def searchEventPattern(row, event_names=None, debug=False):
    s = None
    if 'value' in row.index:
        s = row.loc['value']
    elif 'NvtxEvent.Text' in row.index:
        s = row.loc['NvtxEvent.Text']
    else:
        print('Can search only Names and NVTX dataframes.')
        return False
    for pattern in event_names:
        m = re.search(pattern, s, re.I)
        if m is not None:
            return True
        else:
            if debug:
                print("{} not found in {}".format(pattern, s))
    return False


# Get all rows from DF with the given correlation ID.
# Search in all df columns with 'correlationId' in the name.
def LookupCorrelationID(corrId, df):  # nvtx, cuda, kernels, sync):
    corrid_columns = [
        c for c in df.columns if c.lower().find('correlationid') >= 0
    ]
    dfcorr = None
    for c in corrid_columns:
        df_ = df[df[c] == corrId]
        if dfcorr is None:
            dfcorr = df_
        else:
            dfcorr = dfcorr.append(df_)
    return dfcorr.dropna(axis=1, how='all')


# Convert columns StartNs and EndNs to
# start and end in seconds.
def convertStartEndTimes(df):
    df_ = df.copy()
    df_['start'] = None
    df_['end'] = None
    start_cols = [c for c in df.columns if c.lower().find('startns') >= 0]
    end_cols = [c for c in df.columns if c.lower().find('endns') >= 0]
    for c in start_cols:
        rows = df_[c].notna()
        df_.loc[rows, 'start'] = df_.loc[rows, c] * 10e-10
    for c in end_cols:
        rows = df_[c].notna()
        df_.loc[rows, 'end'] = df_.loc[rows, c] * 10e-10
    return df_


# Get CUDA kernel names for events with the given correlationIDs
def LookupNamebyCorrID(corrid, df, names):
    dfcorr = LookupCorrelationID(corrid, df)
    #     print('{} events with correlation ID {}'.format(dfcorr.shape[0],corrid))
    if dfcorr.shape[0] == 0:
        return []
    namestrings = []
    if 'CudaEvent.kernel.shortName' in dfcorr.columns:
        if dfcorr['CudaEvent.kernel.shortName'].notna().any():
            shortnames = dfcorr['CudaEvent.kernel.shortName']
            shortnames = shortnames[shortnames.notna()].values
            for ID in shortnames:
                try:
                    n = int(ID)
                except:
                    print('Cannot convert {} to int.'.format(n))
                    continue
                namestrings.append(names[names['id'] == n]['value'].values[0])
    return namestrings


# Return rows that contain string
def searchRowsContaining(s, df):
    mask = df.applymap(lambda x: s.lower() in str(x).lower())
    df_ = df[mask.any(axis=1)]
    return df_


# Search events from df within the time range.
# DF must have 'start' and 'end' columns.
def lookupTimeRange(start, end, df):
    startdf = df[df['start'] >= start]
    rangedf = startdf[startdf['end'] <= end]
    return rangedf


# Combine trace evenets within time range and cuda kernels lookup
def lookupAPIandKernelsInTimerange(start, end, traces, kernels, names):
    # Lookup traces (API) events in the given range
    startdf = traces[traces['start'] >= start]
    rangedf = startdf[startdf['end'] <= end]
    # Store results in the DF
    results = pd.DataFrame(columns=[
        'correlationId', 'api_start', 'api_end', 'kernel', 'start', 'end', 'duration'
    ])

    for i, row in rangedf.iterrows():
        # Get correlation ID from the trace event
        corrID = row['TraceProcessEvent.correlationId']
        if corrID == 0:
            continue
        # Get CUDA kernel by correlation ID
        kernel_event = LookupCorrelationID(corrID, kernels)
        if kernel_event is None or kernel_event.shape[0] == 0:
            # No kernels for trace event with the corrID
            continue
        # Get the name of the CUDA kernel
        name = LookupNamebyCorrID(corrID, kernels, names)
        # Append to results DF
        results.loc[results.shape[0]] = [
            corrID, row['start'], row['end'], name[0], kernel_event['start'].values[0],
            kernel_event['end'].values[0], kernel_event['duration'].values[0]
        ]
    return results



# Find NVTX event which encompasses given trace event
def NVTXforAPIevent(trace_event):
    # Start and end in seconds
    start = trace_event.loc['TraceProcessEvent.startNs'] * 10e-10
    end = trace_event.loc['TraceProcessEvent.endNs'] * 10e-10
    # Search NVTX object encompassing events
    nvtxranges = nvtx[nvtx['end'].notna()].copy()
    nvtxranges = nvtxranges[nvtxranges['start'] <= start]
    nvtxranges = nvtxranges[nvtxranges['end'] >= end]
    names = nvtxranges['NvtxEvent.Text'].values
    return names

## Read JSON

In [3]:
json_file = "logs/DL.cont/traces/20200803/nsys_trace_15.json"
data = [json.loads(line) for line in open(json_file, 'r')]

df = json_normalize(data)
int_columns = [
    'NvtxEvent.Timestamp',
    'NvtxEvent.EndTimestamp',
    'CudaEvent.startNs',
    'CudaEvent.endNs',
    'CudaEvent.correlationId',
    'CudaEvent.sync.eventId',
    'TraceProcessEvent.correlationId',
    'TraceProcessEvent.name',
    'TraceProcessEvent.startNs',
    'TraceProcessEvent.endNs',
    'id'
]
for c in int_columns:
    df[c] = df[c].fillna(-1).astype(int).replace(-1, np.nan)
#     df[c] = df[c].astype(int)

df.sample(n=5)

,type,id,value,globalPid,filename,content,Type,CommEvent.Timestamp,CommEvent.GlobalPid,CommEvent.NumOfCpus,...,CudaEvent.kernel.localMemoryTotal,CudaEvent.kernel.gridId,CudaEvent.kernel.registersPerThread,CudaEvent.kernel.sharedMemoryExecuted,CudaEvent.kernel.cacheConfig,CudaEvent.kernel.launched,CudaEvent.kernel.sharedMemoryConfig,globalTid,nameId,priority
12034,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13066,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1310,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10117,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13018,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print('\n'.join(df.columns))

type
id
value
globalPid
filename
content
Type
CommEvent.Timestamp
CommEvent.GlobalPid
CommEvent.NumOfCpus
CommEvent.Command
CommEvent.Commname
CommEvent.PathToExe
CommEvent.WorkDir
CommEvent.Args
CommEvent.Bitness
CommEvent.NsTime
CommEvent.EnvironId
DiagnosticEvent.Timestamp.Time
DiagnosticEvent.Timestamp.Type
DiagnosticEvent.Timestamp.NsTime
DiagnosticEvent.Source
DiagnosticEvent.Level
DiagnosticEvent.Text
DiagnosticEvent.GlobalProcess
TraceProcessEvent.startNs
TraceProcessEvent.endNs
TraceProcessEvent.correlationId
TraceProcessEvent.eventClass
TraceProcessEvent.name
TraceProcessEvent.returnValue
TraceProcessEvent.globalTid
NvtxEvent.Type
NvtxEvent.Timestamp
NvtxEvent.Text
NvtxEvent.GlobalTid
NvtxEvent.EndTimestamp
NvtxEvent.DomainId
NvtxEvent.NsTime
CudaEvent.startNs
CudaEvent.endNs
CudaEvent.correlationId
CudaEvent.deviceId
CudaEvent.contextId
CudaEvent.streamId
CudaEvent.eventClass
CudaEvent.globalPid
CudaEvent.memcpy.sizebytes
CudaEvent.memcpy.copyKind
CudaEvent.memcpy.srcKind
Cu

## Create DF for each type of objects: NVTX, trace, sync, CUDA events and CUDA kernels

In [5]:
# NVTX objects that have NvtxEvent Timestamp
nvtx = df[df['NvtxEvent.Timestamp'].notna()].dropna(axis=1, how='all')
# Convert to seconds as displayed in the Nsight System window
nvtx['start'] = nvtx['NvtxEvent.Timestamp'] * 10e-10
nvtx['end'] = nvtx['NvtxEvent.EndTimestamp'].fillna(0) * 10e-10
nvtx['end'] = nvtx['end'].replace(0,np.nan)

traces = df[df['TraceProcessEvent.startNs'].notna()].dropna(axis=1,how='all')
traces['start'] = traces['TraceProcessEvent.startNs'] * 10e-10
traces['end'] = traces['TraceProcessEvent.endNs'] * 10e-10
traces['duration'] = (traces['TraceProcessEvent.endNs'] - traces['TraceProcessEvent.startNs']) *10e-10


sync = df[df['CudaEvent.sync.eventId'].notna()].dropna(axis=1, how='all')
# Convert to seconds
sync['start'] = sync['CudaEvent.startNs'] * 10e-10
sync['end'] = sync['CudaEvent.endNs'] * 10e-10
sync['duration'] = (sync['CudaEvent.endNs'] - sync['CudaEvent.startNs']) *10e-10

# # CUDA event objects that have Cudaevent startNS
# cuda = df[df['CudaEvent.startNs'].notna()].dropna(axis=1, how='all')
# # Convert to seconds
# cuda['start'] = cuda['CudaEvent.startNs'] * 10e-10
# cuda['end'] = cuda['CudaEvent.endNs'] * 10e-10
# cuda['duration'] = (cuda['CudaEvent.endNs'] - cuda['CudaEvent.startNs']) *10e-10

# CUDA event kernels objects
kernels = df[df['CudaEvent.kernel.shortName'].notna()].dropna(axis=1, how='all')
# Convert to seconds
kernels['start'] = kernels['CudaEvent.startNs'] * 10e-10
kernels['end'] = kernels['CudaEvent.endNs'] * 10e-10
kernels['duration'] = (kernels['CudaEvent.endNs'] - kernels['CudaEvent.startNs']) *10e-10

# Names
names = df[df['value'].notna()].dropna(axis=1, how='all')


print('Names')
display(names.head())
print('NVTX')
display(nvtx.head())
print('traces')
display(traces[traces['duration']>0.00002].head(5))
print(sorted(list(traces['TraceProcessEvent.name'].unique())))
print('sync')
display(sync.head())
# print('CUDA')
# display(cuda.head())
print('kernels')
display(kernels.head())

Names


,type,id,value
0,String,0.0,[Unknown]
1,String,1.0,[kernel.kallsyms]
2,String,2.0,[Max depth]
3,String,3.0,[Broken backtraces]
4,String,4.0,[Called from Java]


NVTX


,Type,NvtxEvent.Type,NvtxEvent.Timestamp,NvtxEvent.Text,NvtxEvent.GlobalTid,NvtxEvent.EndTimestamp,NvtxEvent.DomainId,NvtxEvent.NsTime,NvtxEvent.Color,start,end
285,59.0,59.0,1.814666e+09,Moving model to GPU,282054495340270,5.562507e+09,0,True,NaN,1.814666,5.562507
2215,34.0,34.0,5.563558e+09,Epoch 0 start,282054495340270,NaN,0,True,8.612608e+08,5.563558,NaN
2216,34.0,34.0,5.563566e+09,Training epoch start,282054495340270,NaN,0,True,NaN,5.563566,NaN
2223,59.0,59.0,5.839509e+09,Iteration 1,282054495340270,6.012271e+09,0,True,4.278223e+09,5.839509,6.012271
2233,59.0,59.0,5.841773e+09,FWD pass,282054495340270,5.914782e+09,0,True,4.286513e+09,5.841773,5.914782


traces


,Type,TraceProcessEvent.startNs,TraceProcessEvent.endNs,TraceProcessEvent.correlationId,TraceProcessEvent.eventClass,TraceProcessEvent.name,TraceProcessEvent.returnValue,TraceProcessEvent.globalTid,start,end,duration
272,48.0,7.100969e+08,7.490840e+08,0.0,3.0,34.0,0.0,282054495340270,0.710097,0.749084,0.038987
275,48.0,7.490898e+08,7.799007e+08,0.0,3.0,36.0,0.0,282054495340270,0.749090,0.779901,0.030811
278,48.0,1.086945e+09,1.111344e+09,0.0,3.0,38.0,0.0,282054495340270,1.086945,1.111344,0.024399
283,48.0,1.814632e+09,1.814659e+09,0.0,3.0,39.0,0.0,282054495340270,1.814632,1.814659,0.000027
286,48.0,1.814940e+09,5.508693e+09,228.0,0.0,61.0,0.0,282054495340270,1.814940,5.508693,3.693753


[32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 56.0, 57.0, 58.0, 61.0, 62.0, 63.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 77.0, 141.0, 144.0, 206.0, 207.0, 208.0, 209.0, 212.0, 213.0, 214.0, 215.0, 216.0]
sync


,Type,CudaEvent.startNs,CudaEvent.endNs,CudaEvent.correlationId,CudaEvent.deviceId,CudaEvent.contextId,CudaEvent.streamId,CudaEvent.eventClass,CudaEvent.globalPid,CudaEvent.sync.eventId,CudaEvent.sync.syncType,start,end,duration
925,106.0,5.508879e+09,5.508886e+09,233.0,0.0,1,7,5.0,282054495305728,4.294967e+09,3.0,5.508879,5.508886,0.000008
928,106.0,5.509234e+09,5.509240e+09,242.0,0.0,1,7,5.0,282054495305728,4.294967e+09,3.0,5.509234,5.509240,0.000006
932,106.0,5.509294e+09,5.509299e+09,251.0,0.0,1,7,5.0,282054495305728,4.294967e+09,3.0,5.509294,5.509299,0.000005
936,106.0,5.509353e+09,5.509360e+09,260.0,0.0,1,7,5.0,282054495305728,4.294967e+09,3.0,5.509353,5.509360,0.000007
941,106.0,5.509401e+09,5.509404e+09,269.0,0.0,1,7,5.0,282054495305728,4.294967e+09,3.0,5.509401,5.509404,0.000003


kernels


,Type,CudaEvent.startNs,CudaEvent.endNs,CudaEvent.correlationId,CudaEvent.deviceId,CudaEvent.contextId,CudaEvent.streamId,CudaEvent.eventClass,CudaEvent.globalPid,CudaEvent.kernel.demangledName,...,CudaEvent.kernel.localMemoryTotal,CudaEvent.kernel.gridId,CudaEvent.kernel.registersPerThread,CudaEvent.kernel.sharedMemoryExecuted,CudaEvent.kernel.cacheConfig,CudaEvent.kernel.launched,CudaEvent.kernel.sharedMemoryConfig,start,end,duration
2302,79.0,5.846516e+09,5.846518e+09,3337.0,0.0,1,7,3.0,282054495305728,75,...,51904512.0,1,15.0,0.0,1.0,1.0,0.0,5.846516,5.846518,0.000003
2304,79.0,5.846660e+09,5.847525e+09,3340.0,0.0,1,7,3.0,282054495305728,65,...,51904512.0,2,125.0,32768.0,1.0,1.0,0.0,5.846660,5.847525,0.000865
2306,79.0,5.847527e+09,5.847530e+09,3352.0,0.0,1,7,3.0,282054495305728,78,...,51904512.0,3,18.0,0.0,1.0,1.0,0.0,5.847527,5.847530,0.000003
2311,79.0,5.848590e+09,5.849189e+09,3393.0,0.0,1,7,3.0,282054495305728,128,...,51904512.0,4,32.0,1024.0,1.0,1.0,0.0,5.848590,5.849189,0.000599
2313,79.0,5.849191e+09,5.849571e+09,3401.0,0.0,1,7,3.0,282054495305728,130,...,51904512.0,5,8.0,0.0,1.0,1.0,0.0,5.849191,5.849571,0.000380


## parseOneTrace

In [75]:
# Search Events by pattern
event_name_patterns =  ['.*streamsynchronize'] #,'.*convolutionbackwarddata.*','prediction and loss']
event_names_df = names[names.apply(searchEventPattern, event_names=event_name_patterns,
                                   axis=1)]
event_names_df

,type,id,value
63,String,63.0,cudaStreamSynchronize_v3020


In [76]:
print("Searching NVTX ...")
nvtx_events_df = nvtx[nvtx.apply(searchEventPattern, event_names=event_name_patterns,
                                 axis=1)].copy()
print('Matched Events:')
nvtx_events_df

Searching NVTX ...
Matched Events:


,Type,NvtxEvent.Type,NvtxEvent.Timestamp,NvtxEvent.Text,NvtxEvent.GlobalTid,NvtxEvent.EndTimestamp,NvtxEvent.DomainId,NvtxEvent.NsTime,NvtxEvent.Color,start,end


In [77]:
# Search trace events (cuDNN, cuBLAS API events, CPU side) with the names
# that were found earlier
df_ = traces.copy()
API_events = df_[df_['TraceProcessEvent.name'].isin(event_names_df['id'])].dropna(
    axis=1, how='all')
print("Found {} API events".format(API_events.shape[0]))
API_events.head()

Found 330 API events


,Type,TraceProcessEvent.startNs,TraceProcessEvent.endNs,TraceProcessEvent.correlationId,TraceProcessEvent.eventClass,TraceProcessEvent.name,TraceProcessEvent.returnValue,TraceProcessEvent.globalTid,start,end,duration
923,48.0,5.508877e+09,5.508887e+09,233.0,0.0,63.0,0.0,282054495340270,5.508877,5.508887,0.000009
927,48.0,5.509233e+09,5.509240e+09,242.0,0.0,63.0,0.0,282054495340270,5.509233,5.509240,0.000007
931,48.0,5.509293e+09,5.509299e+09,251.0,0.0,63.0,0.0,282054495340270,5.509293,5.509299,0.000006
935,48.0,5.509353e+09,5.509360e+09,260.0,0.0,63.0,0.0,282054495340270,5.509353,5.509360,0.000007
939,48.0,5.509401e+09,5.509405e+09,269.0,0.0,63.0,0.0,282054495340270,5.509401,5.509405,0.000004


In [63]:
# Store API event names
API_events['name'] = API_events['TraceProcessEvent.name'].apply(
    lambda x: event_names_df[event_names_df['id'] == x]['value'].values[0])

print("Unique API events:")
print(API_events['name'].unique())

# Search NVTX reagons encompassing API events
API_events['NVTX'] = API_events.apply(NVTXforAPIevent, axis=1)
API_events.sample(n=5)

Unique API events:
['cudaStreamSynchronize_v3020' 'cudnnConvolutionBackwardData']


,Type,TraceProcessEvent.startNs,TraceProcessEvent.endNs,TraceProcessEvent.correlationId,TraceProcessEvent.eventClass,TraceProcessEvent.name,TraceProcessEvent.returnValue,TraceProcessEvent.globalTid,start,end,duration,name,NVTX
7501,48.0,6.068720e+09,6.068771e+09,0.0,28.0,50.0,0.0,282054495340364,6.068720,6.068771,0.000051,cudnnConvolutionBackwardData,"[Iteration 2, BWD pass]"
4271,48.0,5.953114e+09,5.953174e+09,0.0,28.0,50.0,0.0,282054495340364,5.953114,5.953174,0.000060,cudnnConvolutionBackwardData,"[Iteration 1, BWD pass]"
6316,48.0,6.040928e+09,6.040994e+09,0.0,28.0,50.0,0.0,282054495340364,6.040928,6.040994,0.000066,cudnnConvolutionBackwardData,"[Iteration 2, BWD pass]"
1673,48.0,5.526812e+09,5.526905e+09,1913.0,0.0,63.0,0.0,282054495340270,5.526812,5.526905,0.000093,cudaStreamSynchronize_v3020,[Moving model to GPU]
6896,48.0,6.055918e+09,6.055984e+09,0.0,28.0,50.0,0.0,282054495340364,6.055918,6.055984,0.000066,cudnnConvolutionBackwardData,"[Iteration 2, BWD pass]"


In [73]:
events = pd.DataFrame(columns=['name', 'NVTX', 'duration', 'GPU side'])

# Search CUDA kernels for API events
# for _, row in API_events.iterrows():
row = API_events.loc[1673]

print(row[['name', 'TraceProcessEvent.correlationId']])
start = row.loc['start']
end = row.loc['end']
duration = end - start
APIname = row['name']
NVTX_arr = row['NVTX']
NVTX_s = ','.join(row['NVTX'])
# Add CPU-side event
events.loc[events.shape[0]] = [row['name'], NVTX_s, duration, False]

# Search CUDA API events in the time range,
# return CUDA kernels
df_ = lookupAPIandKernelsInTimerange(start, end, traces, kernels, names)
print('{} kernels for {:} nvtx:{} ({:.5f}-{:.5f})'.format(
    df_.shape[0], APIname, NVTX_s, df_['start'].min(), df_['end'].max()))
if df_.shape[0] > 0:
    # Execution time of all kernels from the first to the last
    duration = df_['end'].max() - df_['start'].min()
    print('CUDA kernels found by time range. Duration: {:5f}-{:5f}={:5f}'.format(
        df_['end'].max(), df_['start'].min(), duration))
    events.loc[events.shape[0]] = [APIname, NVTX_s, duration, True]
    print("API events and kernels found by time range")
    display(events)

# Search by correlationID
if row['TraceProcessEvent.correlationId'] != 0:
    dfcorr = LookupCorrelationID(row['TraceProcessEvent.correlationId'], df)
    dfcorr = convertStartEndTimes(dfcorr)
    if dfcorr.shape[0] > 0:
        print('dfcorr')
        display(dfcorr)
        try:
            # Leave only CUDA (GPU-side) events
            dfcorr = dfcorr[dfcorr['CudaEvent.startNs'].notna()]
            dfcorr = convertStartEndTimes(dfcorr)

            dfcorr = dfcorr[['CudaEvent.correlationId', 'start', 'end']]
            duration = dfcorr['end'].max() - dfcorr['start'].min()
            print("Events with correlationID {}: duration {:5f}-{:5f}={:5f}".format(
                row['TraceProcessEvent.correlationId'],dfcorr['end'].max(), dfcorr['start'].min(), duration ))

        except:
            print("Exception. No CudaEvent.startNs in ")
            print(dfcorr.columns)

        events.loc[events.shape[0]] = [APIname, NVTX_s, duration, True]

print(
    "API events and kernels found by time range, NVTX ranges and corresponding CUDA kernels"
)
display(events)

name                               cudaStreamSynchronize_v3020
TraceProcessEvent.correlationId                           1913
Name: 1673, dtype: object
0 kernels for cudaStreamSynchronize_v3020 nvtx:Moving model to GPU (nan-nan)
dfcorr


,Type,TraceProcessEvent.startNs,TraceProcessEvent.endNs,TraceProcessEvent.correlationId,TraceProcessEvent.eventClass,TraceProcessEvent.name,TraceProcessEvent.returnValue,TraceProcessEvent.globalTid,CudaEvent.startNs,CudaEvent.endNs,CudaEvent.correlationId,CudaEvent.deviceId,CudaEvent.contextId,CudaEvent.streamId,CudaEvent.eventClass,CudaEvent.globalPid,CudaEvent.sync.eventId,CudaEvent.sync.syncType,start,end
1673,48.0,5.526812e+09,5.526905e+09,1913.0,0.0,63.0,0.0,282054495340270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.52681,5.5269
1674,106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.526813e+09,5.526904e+09,1913.0,0.0,1,7,5.0,282054495305728,4.294967e+09,3.0,5.52681,5.5269


Events with correlationID 1913.0: duration 5.526904-5.526813=0.000091
API events and kernels found by time range, NVTX ranges and corresponding CUDA kernels


,name,NVTX,duration,GPU side
0,cudaStreamSynchronize_v3020,Moving model to GPU,0.000093,False
1,cudaStreamSynchronize_v3020,Moving model to GPU,0.000091,True


In [54]:
# NVTX events
# use_columns = ['NVTX', 'duration', 'start', 'end']
# nvtx_events_df = nvtx_events_df[use_columns]
nvtx_events_df.head()

,Type,NvtxEvent.Type,NvtxEvent.Timestamp,NvtxEvent.Text,NvtxEvent.GlobalTid,NvtxEvent.EndTimestamp,NvtxEvent.DomainId,NvtxEvent.NsTime,NvtxEvent.Color,start,end
3178,59.0,59.0,5.914788e+09,Prediction and loss,282054495340270,5.915680e+09,0,True,4.294902e+09,5.914788,5.915680
6279,59.0,59.0,6.039374e+09,Prediction and loss,282054495340270,6.039682e+09,0,True,4.294902e+09,6.039374,6.039682
10070,59.0,59.0,6.175950e+09,Prediction and loss,282054495340270,6.176267e+09,0,True,4.294902e+09,6.175950,6.176267


In [55]:
nvtx_event = nvtx_events_df.loc[3178]
print(nvtx_event)
# Find encompassing NVTX ranges
nvtxranges =nvtx[nvtx['end'].notna()].copy()
nvtxranges = nvtxranges[nvtxranges['start'] <= nvtx_event['start']]
nvtxranges = nvtxranges[nvtxranges['end'] > nvtx_event['end']]
nvtx_names = ','.join(nvtxranges['NvtxEvent.Text'].values)

# Add NVTX event to events DF
events.loc[events.shape[0]] = [nvtx_event['NvtxEvent.Text'], nvtx_names, duration, False]
display(events.tail())


Type                                       59
NvtxEvent.Type                             59
NvtxEvent.Timestamp               5.91479e+09
NvtxEvent.Text            Prediction and loss
NvtxEvent.GlobalTid           282054495340270
NvtxEvent.EndTimestamp            5.91568e+09
NvtxEvent.DomainId                          0
NvtxEvent.NsTime                         True
NvtxEvent.Color                    4.2949e+09
start                                 5.91479
end                                   5.91568
Name: 3178, dtype: object


,name,NVTX,duration,GPU side
0,cudnnBatchNormalizationForwardTrainingEx,"Iteration 2,FWD pass",0.000022,False
1,cudnnBatchNormalizationForwardTrainingEx,"Iteration 2,FWD pass",0.000027,True
2,Prediction and loss,Iteration 1,0.000022,False


In [74]:
# Find CUDA kernel time (start, end, duration) for each NVTX event
start = nvtx_event['start']
end = nvtx_event['end']
cuda_kernels = lookupAPIandKernelsInTimerange(start, end, traces, kernels, names)
print('CUDA Kernels')
display(cuda_kernels.head())
cuda_start = cuda_kernels['start'].min()
cuda_end = cuda_kernels['end'].max()
duration = cuda_end - cuda_start
print('CUDA times: {:.5f}-{:.5f} ({:.5f}s)'.format(cuda_start, cuda_end,
 duration))
df_cuda = pd.DataFrame(columns=['name', 'NVTX', 'duration', 'GPU side'],
                       data=[[nvtx_event['NvtxEvent.Text'], nvtx_names, duration, True]])
print('CUDA kernels:')
display(df_cuda)
events = events.append(df_cuda, ignore_index=True)
print('Result')
display(events)

CUDA Kernels


,correlationId,api_start,api_end,kernel,start,end,duration
0,8174.0,5.915019,5.915082,kernelTransformReduceInnermostDimIndex,5.915077,5.915107,0.000030
1,8187.0,5.915318,5.915370,cunn_SoftMaxForward,5.915372,5.915383,0.000010
2,8204.0,5.915590,5.915644,cunn_ClassNLLCriterion_updateOutput_kernel,5.915647,5.915653,0.000006


CUDA times: 5.91508-5.91565 (0.00058s)
CUDA kernels:


,name,NVTX,duration,GPU side
0,Prediction and loss,Iteration 1,0.000576,True


Result


,name,NVTX,duration,GPU side
0,cudaStreamSynchronize_v3020,Moving model to GPU,0.000093,False
1,cudaStreamSynchronize_v3020,Moving model to GPU,0.000091,True
2,Prediction and loss,Iteration 1,0.000576,True


In [ ]:
# List CUDA events started after 9.3576s
events_aftertime = kernels[kernels['start']>4.3576].sort_values(by=['start'])
columns = ['Type','CudaEvent.correlationId','CudaEvent.kernel.shortName','start','end','duration']
display(events_aftertime[columns].head(10))

Get name of the kernel with correlation ID 2703

In [ ]:
shortNameID = df[df['CudaEvent.correlationId']==233]['CudaEvent.kernel.shortName'].values[0]
print(shortNameID)

In [ ]:
dfcorr = LookupCorrelationID(row[233], df)

In [ ]:
events = names.loc[names['id'] == shortNameID] #['value'].values
print(events)

In [ ]:
df_ = df[df['id'].notna()].copy()
# df_['id'] = df_['id'].astype(int)
df_.loc[df_['id']==shortNameID]['value']

In [ ]:
corrid_columns = [c for c in df.columns if c.lower().find('correlationid')>=0]
corrid_columns

In [ ]:
ind1 = (df['TraceProcessEvent.correlationId'] == 4070)
count1 = len([i for i in ind1 if i is True])
ind2 = (df['CudaEvent.correlationId'] == 4070)
count2 = len([i for i in ind2 if i is True])
print(count1, count2)


In [ ]:
df_ = None
df_ = df[df['TraceProcessEvent.correlationId'] == 4070].dropna(axis=1,how='all')
display(df_)
df_2 = df[df['CudaEvent.correlationId'] == 4070].dropna(axis=1,how='all')
df_ = pd.merge(df_,df_2, how='outer')
display(df_)

In [ ]:
# All trace events ConvolutionForward
# Find sync event with event ID 37 (cudnnConvolutionForward)
df37 = df[(df['TraceProcessEvent.name']==37)].dropna(axis=1,how='all')
convertStartEndTimes(df37)

In [ ]:
# Search all events with CudaEvent.sync.eventID == 37 : are they related to cudnnConvolutionForward?
df_ = convertStartEndTimes(df[(df['CudaEvent.sync.eventId']==37)])
# Print full 1st row
for c in df_.columns:    
    print('{}: \t{}'.format(c,df_.iloc[0][c]))

### Lookup event Name by correlationID

In [ ]:
corrid = 2703
dfcorr = LookupCorrelationID(corrid,df)
display(dfcorr)
for c in dfcorr.columns:
    print('{}: \t {}, {}'.format(c,dfcorr.iloc[0][c],dfcorr.iloc[1][c]))

In [ ]:
print(dfcorr['CudaEvent.kernel.shortName'].notna().any())
shortnames = dfcorr['CudaEvent.kernel.shortName']
shortnames = shortnames[shortnames.notna()].values
print(shortnames)
for n in shortnames:
    try:
        n = int(n)        
    except:
        print('Cannot convert {} to int.'.format(n))
        continue
    print(names[names['id']==n]['value'].values)
    
display(names[names['id']==106])

In [ ]:
LookupNamebyCorrID(2703, df, names)

#### Find trace event and CUDA event for a given correlation ID

In [ ]:
corrID = 2703
trace_event = LookupCorrelationID(corrID, traces)
kernel_event = LookupCorrelationID(corrID, kernels)
display(trace_event)
display(kernel_event)

### Search rows containing a string

In [ ]:
searchres = searchRowsContaining('generateWinogradTiles', df).dropna(axis=1,how='all')
display(searchres)

### Search trace events within given time range

In [ ]:
# Search the first occurance of convolutionForward
df37 = convertStartEndTimes(df[(df['TraceProcessEvent.name']==37)].dropna(axis=1,how='all'))
display(df37.head(1))
timerange = (df37.iloc[0]['start'],df37.iloc[0]['end'])
print(timerange)

In [ ]:
df_ = lookupTimeRange(timerange[0],timerange[1],traces)
df_

### Search for CUDA kernels corresponding to trace events

In [ ]:
# Store results in the DF
result_kernels = pd.DataFrame(
    columns=['correlationId', 'shortname', 'start', 'end', 'duration'])
# Take corrIDs from the lookup results (from previous block)
for corrID in df_['TraceProcessEvent.correlationId'].unique():
    if corrID == 0:
        continue
    kernel_event = LookupCorrelationID(corrID, kernels)
    name = LookupNamebyCorrID(corrID, df, names)
#     display(kernel_event)
    # Append to results DF
    result_kernels.loc[result_kernels.shape[0]] = [
        corrID, name[0], kernel_event['start'].values[0],
        kernel_event['end'].values[0], kernel_event['duration'].values[0]
    ]
    print('{} {}'.format(corrID,name))
display(result_kernels)

In [ ]:
print(timerange)
results = lookupAPIandKernelsInTimerange(timerange[0], timerange[1], traces, kernels,
                               names)
display(results)

### All ConvolutionForward events by iteration number

In [ ]:
# Find NVTX event which encompasses given trace event
def NVTXforAPIevent(corrID, traces, nvtx):
    events = traces[traces['TraceProcessEvent.correlationId']==corrID]
    if events.shape[0]==0:
        print('No events with correlation ID {}'.format(corrID))
        return None
    start = events['start'].min()
    end = events['start'].max()
#     print('Have {} events from {} till {}'.format(events.shape[0],start,end))
    # Search NVTX object encompassing events
    nvtxranges = nvtx[nvtx['end'].notna()]
    nvtxranges = nvtxranges[nvtxranges['start'] <= start]
    nvtxranges = nvtxranges[nvtxranges['end'] >= end]
    return nvtxranges

In [ ]:
NVTXforAPIevent(9860, traces,nvtx)

## Time of all ConvolutionFoward events

In [ ]:
# Find ID for the given event name
APIevent_name = 'convolutionForward'
APIevent = searchRowsContaining(APIevent_name,names)
# display(APIevent)
APIevent_id = APIevent.iloc[0]['id'].astype(int)
print(APIevent_id)
# Find events with this ID
df_APIevents = traces[traces['TraceProcessEvent.name']==APIevent_id].dropna(axis=1, how='all')
df_APIevents

In [ ]:
def getAPIevent(corrId):
#     print('Got corrID {}'.format(corrId))
    nvtxevent = NVTXforAPIevent(corrId, traces, nvtx)
    return nvtxevent.iloc[0]['NvtxEvent.Text']

In [ ]:
# DF for storing results
# One row for each CUDA kernel
# df_results = pd.DataFrame(columns=['NVTX range','API start','API end','kernel','start','end','duration'])
df_results = None
for i, api_range in df_APIevents.iterrows():
    #     if i > 3400:
    #         break
    #     print(i)
    api_start = api_range['start']
    api_end = api_range['end']
    kernels_ = lookupAPIandKernelsInTimerange(api_start, api_end, traces,
                                              kernels, names)
    kernels_['nvtx'] = kernels_['correlationId'].apply(getAPIevent)
    if df_results is None:
        df_results = kernels_
    else:
        df_results = df_results.append(kernels_, ignore_index=True)
display(df_results.head())

In [ ]:
def GetIterationNumber(nvtx_name):
    s = nvtx_name.replace('Iteration ','')
    try:
        i = int(s)
    except:
        print('Cannot convert {} to int'.format(s))
        return None
    return i

In [ ]:
itertimes = df_results[['duration','nvtx']].groupby(['nvtx'],as_index=False).sum()
itertimes['iteration'] = itertimes['nvtx'].apply(GetIterationNumber)
itertimes.sort_values(by=['iteration'],inplace=True)
itertimes.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=120)
itertimes.plot(x='iteration', y='duration', marker='o', ms=4, mfc='w', ax=ax)
ax.set_ylim(0, 0.016)
ax.grid(ls=':', lw=0.5, alpha=0.9)
ax.set_title('FWD convolution time per itertaion')
ax.set_ylabel('time (s)')
plt.show()

In [ ]:
.
iters